In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import warnings
warnings.filterwarnings('ignore')

In [2]:
nonbiological_mol_IDs = ['NUC', 'ZN', 'CA', 'MG', 'III', 'MN', 'FE', 'CU', 'SF4', 'FE2', 'CO', 'FES', 'GOL', 'NA', 'CL', 'K', 'CU1', 'GOL', 'XE', 'NO2', 'EDO', 'NI', 'BR', 'CD', 'O', 'CS', 'NO', 'TL', 'HG', 'UNL', 'KR', 'SR', 'RB', 'F', 'AG', 'AR', 'U', 'AU', 'MO', 'SE', 'GD', 'YB', 'VX', 'SM', 'LI', 'RE', 'N', 'W', 'OS', 'HO', 'PI', 'EDO', 'PG4', 'OGA', 'SO4', 'HEZ', 'FEO', 'CL', 'DMS', 'ACT', 'MPD', 'GOL', 'NH2', 'CUA', 'SIW', 'PGW', 'IOD', 'BR', '3NI', 'ZRW', '78M', 'UNX', 'MES', 'CCN', 'PO4', 'PEG']

In [3]:
fpgen = AllChem.GetMorganGenerator(radius=2)

# RCSB Data Processing for AlphaFold3 Benchmarking

**By:** Lyu Lab @ The Rockefeller University

**Last updated:** Nov 2025

This notebook outlines the workflow used to process and clean RCSB data in order to extract protein sequences and ligand SMILES for AF3 testing.

*Note: Protein–ligand complex data dated before Sep 30 2021 is designated as the training set, while data from Jan 13 2023 to Dec 15 2024 constitutes the out-of-sample set. Data from Sep 2021 and Jan 2023 was excluded, as it was used for AF3’s evaluation set according to the AF3 paper.*

## 1. Training Set

Using [RCSB](https://www.rcsb.org/), a custom report containing protein-ligand complexes can be generated to obtain all entries released before Sep 30 2021. CSV files that were downloaded in batches need to be combined beforehand.

In [4]:
training_set_df = pd.read_csv('rcsb_training_set.csv')
# training_set_df

Keep entries before Sep 30 2021:

In [5]:
# Convert Release Date to datetime format
training_set_df['Release Date'] = pd.to_datetime(training_set_df['Release Date'], format='%Y-%m-%d').fillna(method='ffill')
training_set_df = training_set_df[training_set_df['Release Date'] <= '2021-09-30']

Obtain unique SMILES from the training set. Remove non-biological ligand IDs and filter out any ions and small fragments. This helps to standardize the Morgan-based Tanimoto coefficient calculations without biases.

In [6]:
training_set_ligs = (training_set_df[['Ligand ID', 'Ligand SMILES']]
                     .dropna()
                     .query('`Ligand ID` not in @nonbiological_mol_IDs')
                     .query('`Ligand SMILES`.str.len() > 6')
                     .set_index('Ligand ID')
                     .drop_duplicates()
                     ['Ligand SMILES']
                    )
training_set_ligs

Ligand ID
SPM                                     C(CCNCCCN)CNCCCN
NT     Cn1cc(cc1C(=O)Nc2cc(n(c2)C)C(=O)NCCC(=N)N)NC(=...
HEM    Cc1c2n3c(c1CCC(=O)O)C=C4C(=C(C5=[N]4[Fe]36[N]7...
NBN                                        CCCC[N+]#[C-]
TNT                c1cc(ccc1C(=N)N)OCCCOc2ccc(cc2)C(=N)N
                             ...                        
RS2     c1ccc(cc1)Oc2ccc(cc2)S(=O)(=O)C3(CCOCC3)CC(=O)NO
IBR                          c1cc(c(cc1N)C=O)CC(=O)OCCBr
ISU                                      [H]N=C(N)[Se]CC
ADU                 CC(=O)NC1C(OC(C1O)N2C=CC(=O)NC2=O)CO
DFR                                 CC1C(C(OC1(CO)O)CO)O
Name: Ligand SMILES, Length: 30695, dtype: object

Save training set ligands, if desired:

In [7]:
# training_set_ligs.to_csv('training_set_ligands.smi', header=False, sep=' ')

## 2. Out-of-Sample Set

### Load Ligand Data

In the RCSB search results, a `Ligand` Tabular Report can be selected to display entries with a release date past Jan 13 2023 and up until Dec 15 2024. The end cutoff was determined by the most recent BioLip2 database update, which was Dec 15 2024 at the time of writing. CSV files that were downloaded in batches will need to be combined beforehand.

In [8]:
test_set_df = pd.read_csv('rcsb_test_set_updated_ligands.csv')
# test_set_df

If there are mutliple ligand entries for a particular protein, the RCSB dataset does not propagate the `Entry ID` after the first ligand entry. These entries can be filled up using the 'ffill' method, where NaN values are replaced with the preceding valid entry. Additionally, any entries that may not be relevant when building the AF3 inputs can be removed.

In [9]:
test_set_df_cleaned = test_set_df.copy()

# Fill in any missing protein information with the preceding valid entry (not NaN)
test_set_df_cleaned['Entry ID'] = test_set_df_cleaned['Entry ID'].fillna(method='ffill')

# Drop any columns not relevant for AF3 testing
test_set_df_cleaned = test_set_df_cleaned.drop(columns=['InChI', 'InChI Key', 'Ligand of Interest', \
                                                        'Ligand Name', 'Asym ID', 'Ligand Formula'])

Use a similar cleaning process that was used for the training set. We make the assumption that very large ligands (MW > 700) are PROTACS/heterobifunctional molecules, and these aren't relevant to our study as we don't predict tricomplexes.

In [10]:
test_set_df_cleaned = (test_set_df_cleaned
                       .dropna(subset=['Ligand SMILES', 'Ligand ID'])
                       .query('`Ligand ID` not in @nonbiological_mol_IDs')
                       .query('`Ligand SMILES`.str.len() > 6')
                       .query('`Ligand MW` < 700')
                      )

# test_set_df_cleaned

Save test set ligands, if desired:

In [11]:
# Pickle it
# test_set_df_cleaned.to_pickle('rcsb_test_set_updated_ligands.pkl')

### Load Sequence Data

A `Sequence` Tabular Report can also be created with the same timeframe. While the `Ligand` report will contain all information pertaining to the ligands in each entry, the `Sequence` report will contain the protein sequence information. This dataset will later be merged with the `Ligand` set and the `BioLip` database.

In [12]:
test_set_sequences_df = pd.read_csv('rcsb_test_set_updated_sequences.csv', usecols=['Entry ID', 'Auth Asym ID', 'Sequence'])

test_set_sequences_df['Auth Asym ID'] = test_set_sequences_df['Auth Asym ID'].str.split(r',\s*')
test_set_sequences_df['Entry ID'] = test_set_sequences_df['Entry ID'].str.lower()

test_set_sequences_df = (test_set_sequences_df
                         .explode('Auth Asym ID')
                         .reset_index(drop=True)
                         .rename(columns={'Entry ID': 'pdb_id', 'Auth Asym ID': 'receptor_chain'})
                        )
# test_set_sequences_df

## 3. BioLip2 Database

The data contained in the BioLip2 txt file can be merged with the out-of-sample data to ensure our inputs for testing are only protein-ligand complexes.

In [13]:
biolip_df = pd.read_csv('BioLiP.txt', sep='\t', usecols=['pdb_id', 'receptor_chain', 'ligand_chain'])
# biolip_df

## 4. Novelty Calculation

The Tanimoto coefficient (Tc) can be calculated for each training-test ligand pair, with the maximum value getting saved.

### Generate Morgan Fingerprints

First, generate Morgan fingerprints from the SMILES strings in the training set using RDKit. Ignore SMILES that have trouble getting processed by RDKit.

In [14]:
training_set_mols = {ID: Chem.MolFromSmiles(smi) for ID, smi in training_set_ligs.items() if Chem.MolFromSmiles(smi)}
# training_set_mols

In [15]:
training_set_fps = fpgen.GetFingerprints(list(training_set_mols.values()))
# training_set_fps

Next, generate Morgan fingerprints of the SMILES strings in the test set.

In [16]:
test_set_df_cleaned['RDKit Mol'] = test_set_df_cleaned['Ligand SMILES'].apply(Chem.MolFromSmiles)

# Drop unsanitized ligands
test_set_df_cleaned = test_set_df_cleaned.dropna(subset=['RDKit Mol'])

# Generate Morgan fingerprints
test_set_df_cleaned['Morgan Fingerprint'] = test_set_df_cleaned['RDKit Mol'].apply(fpgen.GetFingerprint)

### Calculate Tc

In [17]:
def get_tc_scores(test_set_fp, fps=training_set_fps) -> list:
    """Generate Tc matrix for a ligand in the test set compared to the training set database."""
    return DataStructs.BulkTanimotoSimilarity(test_set_fp, fps)

In [18]:
# Calculate Tc (each test ligand to all ligands in training set)
test_set_df_cleaned['Tanimoto Scores'] = test_set_df_cleaned['Morgan Fingerprint'].apply(get_tc_scores)

In [19]:
# Obtain max Tc score for each test ligand
test_set_df_cleaned['max_tanimoto'] = test_set_df_cleaned['Tanimoto Scores'].apply(max)

### Merge and filter test set

In [20]:
# Split `Auth Asym ID` by comma
test_set_df_cleaned['Auth Asym ID'] = test_set_df_cleaned['Auth Asym ID'].str.split(r',\s*')

# Change `Entry ID` to lowercase
test_set_df_cleaned['Entry ID'] = test_set_df_cleaned['Entry ID'].str.lower()

# Clean up DataFrame
test_set_df_cleaned = (test_set_df_cleaned
                       .explode('Auth Asym ID')
                       .reset_index(drop=True)
                       .rename(columns={'Auth Asym ID': 'ligand_chain', 'Entry ID': 'pdb_id'})
                       .drop(columns=['Ligand MW', 'RDKit Mol', 'Morgan Fingerprint', 'Tanimoto Scores'])
                      )

In [21]:
# Merge with Sequence and Biolip DataFrames
test_set_df_cleaned = (test_set_df_cleaned
                       .merge(biolip_df, on=['pdb_id', 'ligand_chain'], how='inner')
                       .merge(test_set_sequences_df, on=['pdb_id', 'receptor_chain'], how='inner')
                       .drop_duplicates(subset=['Ligand SMILES', 'Sequence'], keep='first')
                       .rename(columns={'Ligand ID': 'ligand_id', 'Ligand SMILES': 'ligand_smiles', 'Sequence': 'receptor_sequence'})
                      )

In [22]:
test_set_df_cleaned

,pdb_id,ligand_chain,ligand_id,ligand_smiles,max_tanimoto,receptor_chain,receptor_sequence
0,5sml,A,O3R,CN(Cc1ccc(c(c1)Cl)Cl)c2ccc(cn2)S(=O)(=O)N,0.411765,A,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
1,5smm,A,O46,c1cc(cc(c1)O)CC(=O)NC2(CCOCC2)c3cccc(c3)F,0.387755,A,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
2,5smn,A,O4F,Cc1cnccc1N2CCC(CC2)C(=O)NC3(CC3)C#N,0.402985,A,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
3,5sn5,A,Q40,c1ccc(cc1)CCN2CCC(CC2)C(=O)N,0.531915,A,MSRRRVVITGMGMLSPLGLDVPSSWEGILAGRSGIAPIEHMDLSAY...
4,5sn6,A,EJQ,c1cc(ccc1NC(=O)CN2CCCC2)F,1.000000,A,MSRRRVVITGMGMLSPLGLDVPSSWEGILAGRSGIAPIEHMDLSAY...
...,...,...,...,...,...,...,...
89005,9k1d,R,BUA,CCCC(=O)O,1.000000,R,MKTIIALSYIFCLVFADYKDDDDWSHPQFEKGGGSGGGSGGSAWSH...
89010,9k1d,R,CLR,CC(C)CCCC(C)C1CCC2C1(CCC3C2CC=C4C3(CCC(C4)O)C)C,1.000000,R,MKTIIALSYIFCLVFADYKDDDDWSHPQFEKGGGSGGGSGGSAWSH...
89015,9k1j,A,A1EEB,c1cc2c(cc1F)SNC2=O,0.405405,A,MSTLYSTQVKAVGGRSGTIRSEDGILELKLALPKELGGKGDATNPE...
89019,9k3q,1,CRT,CC(=CC=CC(=CC=CC(=CC=CC=C(C)C=CC=C(C)C=CC=C(C)...,1.000000,1,ITEGEAKEFHKIFTSSILVFFGVAAFAHLLVWIWRPWVPGPNGY


Save csv, to be used to create AF3 json inputs:

In [23]:
# test_set_df_cleaned.to_csv('af3_inputs.csv', index=False)